# Hashing trick. Библиотека Vowpal Wabbit

### Hashing Trick
One hot encoding — просто и понятно, но 
1) дорого по ресурсам
2) может быть вычислительно неусточиво из-за большого числа категорий


Давайте вместо этого от признака возьмем его Hash по некоторому модулю N. Тогда для каждого категориального признака будет N значений

Spam dataset https://www.kaggle.com/datasets/venky73/spam-mails-dataset/

In [5]:
import pandas as pd

In [8]:
data = pd.read_csv('../../data/spam/spam_ham_dataset.csv', index_col=0)

In [12]:
data.loc[:, 'email_text'] = data['text'].apply(lambda x: x.split('\n', 1)[1])

In [33]:
data.head()

,label,text,label_num,email_text,clean_text
605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,this is a follow up to the note i gave you on ...,Subject enron methanol meter 988291 this is a ...
2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,( see attached file : hplnol 09 . xls )\r\n- h...,Subject hpl nom for january 9 2001 see attache...
3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,"ho ho ho , we ' re around to that most wonderf...",Subject neon retreat ho ho ho we re around to ...
4685,spam,"Subject: photoshop , windows , office . cheap ...",1,abasements darer prudently fortuitous undergon...,Subject photoshop windows office cheap main tr...
2030,ham,Subject: re : indian springs\r\nthis deal is t...,0,this deal is to book the teco pvr revenue . it...,Subject re indian springs this deal is to book...


In [17]:
import re

def clean_text(text):
    clean = []
    for c in text:
        if c.isalnum() or c.isspace():
            clean.append(c)
            
    clean_text = ''.join(clean)
    return re.sub('\\s+', ' ', clean_text)

In [20]:
data.loc[:, 'clean_text'] = data['text'].apply(clean_text)

### Формат данных для WV

target | f_1 f_2 f_3 f_3:w_4 | g_1 g_2 g_3 

In [25]:
def data_to_vw_format(data_frame, path):
    with open(path, 'w', encoding='utf-8') as output:
        for _, row in data_frame.iterrows():
            target = row['label_num']
            features = row['clean_text']
            output.write(str(target))
            output.write(' | ')
            output.write(features)
            output.write('\n')

In [26]:
n_rows = len(data)
train_size = int(0.75 * n_rows)

train = data.iloc[:train_size]
test = data.iloc[train_size:]

In [27]:
data_to_vw_format(train, 'train.txt')
data_to_vw_format(test, 'test.txt')

In [28]:
!head train.txt

0 | Subject enron methanol meter 988291 this is a follow up to the note i gave you on monday 4 3 00 preliminary flow data provided by daren please override pop s daily volume presently zero to reflect daily activity you can obtain from gas control this change is needed asap for economics purposes 
0 | Subject hpl nom for january 9 2001 see attached file hplnol 09 xls hplnol 09 xls
0 | Subject neon retreat ho ho ho we re around to that most wonderful time of the year neon leaders retreat time i know that this time of year is extremely hectic and that it s tough to think about anything past the holidays but life does go on past the week of december 25 through january 1 and that s what i d like you to think about for a minute on the calender that i handed out at the beginning of the fall semester the retreat was scheduled for the weekend of january 5 6 but because of a youth ministers conference that brad and dustin are connected with that week we re going to change the date to the follow

In [34]:
!vw -d train.txt -f vw_model.bin --loss_function logistic

final_regressor = vw_model.bin
using no cache
Reading datafile = train.txt
num sources = 1
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
Enabled learners: gd, scorer-identity, count_label
Input label = SIMPLE
Output pred = SCALAR
average  since         example        example        current        current  current
loss     last          counter         weight          label        predict features
0.693147 0.693147            1            1.0         0.0000         0.0000       54
0.693147 0.693147            2            2.0         0.0000         0.0000       17
0.693147 0.693147            4            4.0         1.0000         0.0000       45
0.692047 0.690947            8            8.0         1.0000         0.0203      127
0.596534 0.501020           16           16.0         1.0000        16.3499     1423
0.661809 0.727083           32           32.0         0.0000         0.2072       26
0.827191 0.992573           64           64.0         0.0000       